In [21]:
import json
from io import StringIO
import mmap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import swmmio

np.warnings.filterwarnings('ignore')
%matplotlib inline

In [22]:
with open('../data/orifice.json', 'r') as j:
    inp = json.load(j)
    
superjunctions = pd.DataFrame(inp['superjunctions'])
superlinks = pd.DataFrame(inp['superlinks'])
junctions = pd.DataFrame(inp['junctions'])
links = pd.DataFrame(inp['links'])
orifices = pd.DataFrame(inp['orifices'])

In [23]:
model = swmmio.Model('/Users/mdbartos/Downloads/aa_0360min_025yr.inp')

In [24]:
junctionmap = pd.Series(np.arange(len(model.nodes())), model.nodes().index.values)
linkmap = pd.Series(np.arange(len(model.conduits())), model.conduits().index.values)

In [25]:
tabular = pd.Series(junctionmap[11:].index, junctionmap[11:].values) 

In [26]:
new_superjunctions = pd.DataFrame(np.zeros((model.nodes().shape[0], superjunctions.shape[1])))
new_superjunctions.columns = superjunctions.columns

new_superjunctions['storage'] = 'functional'
new_superjunctions['z_inv'] = model.nodes()['InvertElev'].values
new_superjunctions['z_inv'] = 0.3048 * (new_superjunctions['z_inv'] - new_superjunctions['z_inv'].min())
new_superjunctions['id'] = new_superjunctions.index.values
new_superjunctions['table'] = tabular
new_superjunctions.loc[tabular.index, 'storage'] = 'tabular'
new_superjunctions.loc[new_superjunctions['storage'] == 'functional', 'c'] = 1

In [27]:
new_superjunctions

,A_sj,a,b,bc,c,h_0,id,storage,z_inv,table
0,0.0,0.0,0.0,0.0,1.0,0.0,0,functional,23.826216,NaN
1,0.0,0.0,0.0,0.0,1.0,0.0,1,functional,21.640800,NaN
2,0.0,0.0,0.0,0.0,1.0,0.0,2,functional,18.592800,NaN
3,0.0,0.0,0.0,0.0,1.0,0.0,3,functional,19.574256,NaN
4,0.0,0.0,0.0,0.0,1.0,0.0,4,functional,14.715744,NaN
5,0.0,0.0,0.0,0.0,1.0,0.0,5,functional,21.031200,NaN
6,0.0,0.0,0.0,0.0,1.0,0.0,6,functional,17.678400,NaN
7,0.0,0.0,0.0,0.0,1.0,0.0,7,functional,10.972800,NaN
8,0.0,0.0,0.0,0.0,1.0,0.0,8,functional,6.858000,NaN
9,0.0,0.0,0.0,0.0,1.0,0.0,9,functional,3.505200,NaN


In [28]:
new_superlinks = pd.DataFrame(np.zeros((model.conduits().shape[0], superlinks.shape[1])))
new_superlinks.columns = superlinks.columns

new_superlinks['sj_0'] = model.conduits()['InletNode'].map(junctionmap).values
new_superlinks['sj_1'] = model.conduits()['OutletNode'].map(junctionmap).values
new_superlinks = new_superlinks.sort_values(['sj_0', 'sj_1']).reset_index(drop=True)
j_0 = 4 * np.arange(0, new_superlinks.shape[0])
j_1 = 3 + j_0
new_superlinks['j_0'] = j_0
new_superlinks['j_1'] = j_1
new_superlinks['id'] = new_superlinks.index.values

In [29]:
new_superlinks

,id,j_0,j_1,sj_0,sj_1
0,0,0,3,0,21
1,1,4,7,1,19
2,2,8,11,2,13
3,3,12,15,3,15
4,4,16,19,4,14
5,5,20,23,5,19
6,6,24,27,6,14
7,7,28,31,7,16
8,8,32,35,8,18
9,9,36,39,9,17


In [30]:
new_links = pd.DataFrame(np.zeros((3 * model.conduits().shape[0], links.shape[1])))
new_links.columns = links.columns
new_links['ctrl'] = False
new_links['k'] = np.repeat(new_superlinks.index.values, 3)

lengths = 0.3048 * model.conduits()['Length']

first_lengths = np.minimum(0.1*lengths, 10)
last_lengths = np.minimum(0.1*lengths, 10)
remaining_lengths = lengths - first_lengths - last_lengths
assert ((remaining_lengths + first_lengths + last_lengths) == lengths).all()
link_lengths = np.column_stack([first_lengths, remaining_lengths, last_lengths]).ravel()
new_links['dx'] = link_lengths
new_links['shape'] = 'circular'
new_links['n'] = model.conduits().iloc[new_links['k']].loc[:, 'ManningN'].values
new_links['g1'] = model.conduits().iloc[new_links['k']].loc[:, 'Geom1'].values

j_0 = np.arange(4 * len(new_superlinks))
j_1 = np.arange(4 * len(new_superlinks))
jj_01 = np.column_stack([j_0[:-1], j_1[1:]])
jj_01 = jj_01[np.delete(np.arange(len(jj_01)), np.arange(3, len(jj_01), 4))]
new_links[['j_0', 'j_1']] = jj_01
new_links['id'] = new_links.index.values

In [31]:
new_links

,A_c,C,Q_0,ctrl,dx,g1,g2,g3,id,j_0,j_1,k,n,shape
0,0.0,0.0,0.0,False,10.000000,1.0,0.0,0.0,0,0,1,0,0.010,circular
1,0.0,0.0,0.0,False,319.434424,1.0,0.0,0.0,1,1,2,0,0.010,circular
2,0.0,0.0,0.0,False,10.000000,1.0,0.0,0.0,2,2,3,0,0.010,circular
3,0.0,0.0,0.0,False,5.634533,1.0,0.0,0.0,3,4,5,1,0.010,circular
4,0.0,0.0,0.0,False,45.076262,1.0,0.0,0.0,4,5,6,1,0.010,circular
5,0.0,0.0,0.0,False,5.634533,1.0,0.0,0.0,5,6,7,1,0.010,circular
6,0.0,0.0,0.0,False,10.000000,1.0,0.0,0.0,6,8,9,2,0.010,circular
7,0.0,0.0,0.0,False,263.845000,1.0,0.0,0.0,7,9,10,2,0.010,circular
8,0.0,0.0,0.0,False,10.000000,1.0,0.0,0.0,8,10,11,2,0.010,circular
9,0.0,0.0,0.0,False,5.561381,1.0,0.0,0.0,9,12,13,3,0.010,circular


In [32]:
new_junctions = pd.DataFrame(np.zeros((4 * model.conduits().shape[0], junctions.shape[1])))
new_junctions.columns = junctions.columns
new_junctions['id'] = new_junctions.index.values
new_junctions['k'] = np.repeat(new_superlinks.index.values, 4)
dz_k = (new_superjunctions.loc[new_superlinks['sj_0'].values, 'z_inv'].values
        - new_superjunctions.loc[new_superlinks['sj_1'].values, 'z_inv'].values)
dx_k = new_links.groupby('k').sum()['dx'].values
# Remember to do unit conversion
dzdx = pd.Series(dz_k / dx_k)
drop = new_links['k'].map(dzdx) * new_links.groupby('k').cumsum()['dx']
first_depth = new_superjunctions.loc[new_superlinks['sj_0'].loc[new_links['k'].values].values, 'z_inv']
drop.index = new_links['j_1'].loc[drop.index.values]
first_depth.index = drop.index
z_inv = first_depth - drop
new_junctions.loc[z_inv.index, 'z_inv'] = z_inv

first_depth = new_superjunctions.loc[new_superlinks['sj_0'].loc[new_links['k'].values].values, 'z_inv']
first_depth.index = new_superlinks['j_0'].loc[new_links['k'].values].values
first_depth = first_depth.groupby(level=0).first()
new_junctions.loc[first_depth.index, 'z_inv'] = first_depth.values
new_junctions['A_s'] = new_junctions['k'].map(new_links.groupby('k').mean()['g1']).values

In [33]:
new_junctions

,A_s,h_0,id,k,z_inv
0,1.0,0.0,0,0,2.382622e+01
1,1.0,0.0,1,0,2.376183e+01
2,1.0,0.0,2,0,2.170518e+01
3,1.0,0.0,3,0,2.164080e+01
4,1.0,0.0,4,1,2.164080e+01
5,1.0,0.0,5,1,2.133600e+01
6,1.0,0.0,6,1,1.889760e+01
7,1.0,0.0,7,1,1.859280e+01
8,1.0,0.0,8,2,1.859280e+01
9,1.0,0.0,9,2,1.856059e+01


In [34]:
new_orifices = pd.DataFrame(np.zeros((model.orifices().shape[0], orifices.shape[1])))
new_orifices.columns = orifices.columns

new_orifices['id'] = new_orifices.index.values
new_orifices['sj_0'] = model.orifices()['InletNode'].map(junctionmap).values
new_orifices['sj_1'] = model.orifices()['OutletNode'].map(junctionmap).values
new_orifices['A'] = 1.

In [35]:
new_orifices

,A,id,sj_0,sj_1
0,1.0,0,12,0
1,1.0,1,21,1
2,1.0,2,19,2
3,1.0,3,15,4
4,1.0,4,11,3
5,1.0,5,20,5
6,1.0,6,13,6
7,1.0,7,14,7
8,1.0,8,16,8
9,1.0,9,18,9


In [36]:
with open('/Users/mdbartos/Downloads/aa_0360min_025yr.inp', 'r') as f:
    s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    begin = s.find(b'[CURVES]')
    end = s[begin:].find(b'[TIMESERIES]')
    curves = s[begin:begin+end].decode()

In [37]:
curves = curves[curves.find('--\n')+3:]

In [38]:
split = curves.split('\n\n')

In [39]:
storages = {}

for elem in split:
    try:
        storage = pd.read_csv(StringIO(elem), sep='\t', header=None)
        name = storage[0].str.strip()[0]
        h = 0.3048 * storage[2].values
        A = 0.3048**2 * storage[3].values
        storages[name] = {}
        storages[name]['h'] = h
        storages[name]['A'] = A
    except:
        print(elem)

In [40]:
with open('/Users/mdbartos/Downloads/aa_0360min_025yr.inp', 'r') as f:
    s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    begin = s.find(b'[STORAGE]')
    end = s[begin:].find(b'\n\n')
    table = s[begin:begin+end].decode()

In [41]:
# Storage names are same as storage id
pd.read_csv(StringIO('\n'.join(table.split('\n')[4:])), sep='\t', header=None)

,0,1,2,3,4,5,6,7,8,9
0,93-49743,874.22,14.96,0.0,TABULAR,93-49743,NaN,,32993.0,0.0
1,93-49868,888.17,10.00,0.0,TABULAR,93-49868,NaN,,50000.0,0.0
2,93-49919,868.00,10.00,0.0,TABULAR,93-49919,NaN,,50000.0,0.0
3,93-49921,846.00,10.00,0.0,TABULAR,93-49921,NaN,,50000.0,0.0
4,93-50074,858.28,13.72,0.0,TABULAR,93-50074,NaN,,50000.0,0.0
5,93-50076,832.50,10.00,0.0,TABULAR,93-50076,NaN,,50000.0,0.0
6,93-50077,810.00,10.00,0.0,TABULAR,93-50077,NaN,,50000.0,0.0
7,93-50081,821.50,10.00,0.0,TABULAR,93-50081,NaN,,50000.0,0.0
8,93-50225,871.00,20.00,0.0,TABULAR,93-50225,NaN,,36888.0,0.0
9,93-90357,879.00,10.00,0.0,TABULAR,93-90357,NaN,,50000.0,0.0
